# <center>PRÁCTICA 1: APRENDIZAJE NO SUPERVISADO (DengAI)<center>

**Nombre y apellidos:**

**Usuario VIU:**

<font color="green">*\[\[IMPORTANTE: Este notebook debe usarse como plantilla para empezar esta práctica. Antes de entregar el notebook, se deben eliminar todas las líneas de texto entre dobles corchetes como esta.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Se debe leer y comprender esta plantilla de principio a fin antes de empezar. En estas líneas con anotaciones se resuelven las dudas y errores más comunes de la elaboración de esta práctica.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Todas las secciones definidas en esta plantilla deben estar presentes en la entrega final y respetarse para facilitar la corrección, aunque cada estudiante puede añadir las subsecciones que considere. Los huecos para rellenar al principio del notebook y la tabla de la sección "Conclusiones" son obligatorios.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Se debe entregar un único notebook de Jupyter (fichero *.ipynb) con la práctica desarrollada. Debe estar autocontenido, ya que no se corregirá nada más que este fichero entregado.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Todo el notebook debe ser ejecutable. En el notebook se deben dejar los resultados de los experimentos, reproducibles por el profesor en su sesión de Google Colab. Se aconseja encarecidamente que se fije una semilla aleatoria única para todos los procesos estocásticos que se vayan a llevar a cabo.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: El fichero *.ipynb, antes de corregirse, se pasará por un detector de plagios específico para estos notebooks de Jupyter con el fin de evitar cualquier tipo de fraude. Los resultados de este detector se revisarán manualmente por lo que se sugiere que el alumno especifique la fuente de los algoritmos (si son de la web) a fin de evitar problemas (no hará falta si son pequeños fragmentos de otras prácticas en las que participó el mismo alumno). Si se demuestra copia de toda o parte de esta práctica de otros alumnos de esta asignatura, en la convocatoria/edición de la asignatura actual o cualquiera anterior de la asignatura, implicará un suspenso con 0 para todas las partes implicadas y la apertura de un expediente.\]\]*</font>

---
# Resumen
---

<font color="green">*\[\[Describir qué se va a hacer en esta práctica, mencionar las diferentes fases en las que se ha dividido la experimentación, así como las técnicas que se van a explicar en las siguientes secciones. Resumir también, muy brevemente, los principales resultados obtenidos.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Será necesario registrarse en la competición de DengAI de DrivenData para poder descargar el dataset: https://www.drivendata.org/competitions/44/dengai-predicting-disease-spread/ \]\]*</font>

---
# Inicialización
---

In [ ]:
# Imports generales
import pandas as pd
import io
from google.colab import files

seed = 42  # Semilla aleatoria arbitraria y constante a incluir en los algoritmos estocásticos para que los experimentos sean siempre reproducibles por el profesor.

def upload_files (index_fields=None):
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), index_col = index_fields)
    return df

In [ ]:
# Subir el conjunto de entrenamiento sin variable objetivo (dengue_features_train.csv)
train = upload_files()
print(train.shape)
train.head()

---
# Exploración
---

<font color="green">*\[\[En esta y las siguientes secciones se irán describiendo y ejecutando los distintos experimentos, intercalando celdas de texto y código.\]\]*</font>

<font color="green">*\[\[Aquí se espera un análisis estadístico del dataset, generar características derivadas adicionales (si procede), detección de outliers y su eliminación (si procede), etc. El objetivo es tener una idea general de cómo son los datos, cómo están distribuidos los datos, etc.\]\]*</font>

<font color="green">*\[\[Una vez hecho eso, será necesario preprocesar esos datos, limpiarlos, detectar outliers y eliminarlos (si procede) etc. Deberían incluirse todas las funciones y transformaciones que permitirán el uso de los algoritmos de ML sobre el dataset.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: En este enlace se puede ver una descripción de qué significan las características del dataset: https://www.drivendata.org/competitions/44/dengai-predicting-disease-spread/page/82/ \]\]*</font>

<font color="green">*\[\[IMPORTANTE: Todos los gráficos que se representen en este notebook deben indicar claramente el título de cada eje de coordenadas.\]\]*</font>

In [ ]:
# EJEMPLO: Eliminar variable week_start_date por ser de tipo cadena de texto sin formato
train.drop("week_start_date", axis = 1, inplace = True)

<font color="green">*\[\[Rellenar valores perdidos. Hacerlo de la forma más inteligente posible podría mejorar mucho los resultados. Aquí solo se ilustra la forma más básica.\]\]*</font>

In [ ]:
# EJEMPLO: Contar valores perdidos
train.isnull().sum()

In [ ]:
# EJEMPLO: Completar valores perdidos con método ffill
# Alternativa: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate
train.fillna(method='ffill', inplace=True)
pd.isnull(train).any()

<font color="green">*\[\[Transformar variables categóricas si hace falta.\]\]*</font>

In [ ]:
# EJEMPLO: Binarizador para la variable categórica city en el conjunto de entrenamiento y test
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
train['city_bin'] = lb.fit_transform(train['city'])

train.tail()

---
# Características
---

<font color="green">*\[\[IMPORTANTE: Si se hace un análisis con el dataset traspuesto para investigar la selección de características, no olvidar también la experimentación con el dataset sin trasponer (incluído clústering, outliers, etc.) de otras secciones.\]\]*</font>

<font color="green">*\[\[El resultado final de esta sección será un nuevo conjunto de características que podría incluir una o más de los siguientes tipos: originales sin modificar, originales con alguna transformación aplicada y características nuevas derivadas de la combinación de otras originales. Técnicamente se podrían incluir todas las características en los experimentos, pero aquí se va a evaluar la capacidad del estudiante de encontrar correlaciones y determinar cuáles podrían eliminarse o no para facilitar el trabajo de los algoritmos de machine learning.\]\]*</font>

<font color="green">*\[\[En aprendizaje no supervisado es conveniente normalizar los datos. No obstante, la necesidad de normalizar depende del problema que se esté tratando y de la propia naturaleza de las características. Hay ciertos casos en los que, dependiendo de cómo se aplique, ésta pueda llegar a ser perjudicial o no efectiva.\]\]*</font>

In [ ]:
# EJEMPLO: Conjunto -aleatorio- de características solo para ilustrar los siguientes pasos
# IMPORTANTE: no tomar esta selección aleatoria como referencia
features = ['reanalysis_specific_humidity_g_per_kg', 'reanalysis_dew_point_temp_k', 'station_avg_temp_c', 'station_min_temp_c']

In [ ]:
# EJEMPLO: Normalización con MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(train[features])
X_train

---
# Clustering
---

<font color="green">*\[\[En esta sección se busca que el estudiante pruebe varias técnicas de clustering sobre el dataset, así como que las compare entre sí y decida cuál de ellas considera que es la que ha logrado agrupar mejor los datos en base a distintas métricas, experimentos, etc.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Hay que comparar con más de una técnica de clústering, como mínimo. Como aportación adicional, también se pueden usar algoritmos no vistos en clase.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Se esperan clusterings de más de 3 dimensiones, aunque para visualizar los resultados en 2D/3D se pueden utilizar técnicas de reducción de dimensionalidad.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: No basta con ejecutar los algoritmos y mostrar el resultado. Hay que comparar lo obtenido con los experimentos que se han hecho e ir extrayendo deducciones acerca de por qué un curso de experimentación fue o será mejor que otro.\]\]*</font>

---
# Conclusiones
---

<font color="green">*\[\[Discusión final de los resultados de los resultados expuestos en la memoria, justificando la selección final de del algoritmo de clustering y de las variables finales.\]\]*</font>

<font color="green">*\[\[Discutir trabajo futuro: Mejoras del trabajo realizado que podrían llevarse a cabo pero que no se consideran parte de esta entrega.\]\]*</font>